In [4]:
import pandas as pd

# Carga el dataset enriquecido (los datos crudos)
raw_enriched_path = "../../data/ridership_enriched_final.parquet"
df = pd.read_parquet(raw_enriched_path)

print(f"Número de filas crudas a procesar: {len(df)}")

# --- ¡LA SOLUCIÓN! ---
# 1. Filtramos las filas "placeholder" que no tienen actividad.
#    Nos quedamos solo con los registros donde al menos un pasajero subió.
df_filtered = df[df['ridership'] > 0].copy()
print(f"Número de filas con actividad real: {len(df_filtered)}")

# 2. Crea la columna de timestamp redondeada a la hora en el DF filtrado
df_filtered['hour_timestamp'] = df_filtered['transit_timestamp'].dt.floor('h')

# 3. Agrupa el DF FILTRADO
df_agg = df_filtered.groupby(['bus_route', 'hour_timestamp'], observed=False).agg(
    ridership_total=('ridership', 'sum'),
    transfers_total=('transfers', 'sum'),
    operating_buses=('bus_route', 'count')
).reset_index()

# 4. Une las variables enriquecidas
enriched_cols = [
    'hour', 'day_of_week', 'month', 'year', 'is_weekend', 'is_holiday',
    'temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain',
    'snowfall', 'weather_code', 'wind_speed_10m'
]
df_enriched_unique = df_filtered.drop_duplicates(subset=['bus_route', 'hour_timestamp'])
df_model_input = pd.merge(
    df_agg,
    df_enriched_unique[['bus_route', 'hour_timestamp'] + enriched_cols],
    on=['bus_route', 'hour_timestamp']
)

# 5. Ordena y analiza la nueva y correcta columna 'operating_buses'
df_model_input.sort_values(by=['bus_route', 'hour_timestamp'], inplace=True)
print("\n--- ANÁLISIS FINAL de la nueva columna 'operating_buses' ---")
print(df_model_input['operating_buses'].describe())
print("\n--- Frecuencia de Valores (Top 10) ---")
print(df_model_input['operating_buses'].value_counts().head(10))

# 6. Guarda el DataFrame final
model_input_path = "../../data/model_input.parquet"
df_model_input.to_parquet(model_input_path, index=False)
print(f"\n✅ Dataset final para el modelo (CORREGIDO) guardado en: {model_input_path}")

Número de filas crudas a procesar: 630720
Número de filas con actividad real: 148331

--- ANÁLISIS FINAL de la nueva columna 'operating_buses' ---
count    19632.000000
mean         7.555573
std          1.818623
min          1.000000
25%          7.000000
50%          8.000000
75%          9.000000
max         10.000000
Name: operating_buses, dtype: float64

--- Frecuencia de Valores (Top 10) ---
operating_buses
9     7225
8     5035
7     3515
5     1028
6      999
4      525
3      417
10     307
1      296
2      285
Name: count, dtype: int64

✅ Dataset final para el modelo (CORREGIDO) guardado en: ../../data/model_input.parquet
